In [2]:
import sys
sys.executable

'd:\\Code-Projects\\University\\Fall2023\\Grad\\LipNet-Assistive-AI\\.conda\\python.exe'

In [34]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from time import sleep
import pysrt
import re
import string
import subprocess
import glob
import shutil


In [58]:
# Function to download video
def download_video(url, id):
        try:
            yt = YouTube(url)

            stream = (
                yt.streams.filter(progressive=True, file_extension="mp4").get_highest_resolution()
            )

            if "a.ar" in yt.captions.keys():
                captions = yt.captions.get_by_language_code("a.ar")
                captions.download(
                    srt=True,
                    output_path=r"data/Video Links Dataset/temp_alignments",
                    title=f"{id}")

                stream.download(
                    filename=fr"data/Video Links Dataset/temp_videos/{id}.mp4"
                )

                return fr"data/Video Links Dataset/temp_videos/{id}.mp4",fr"data/Video Links Dataset/temp_alignments/{id} (a.ar).srt", id

            else:
                print("skipping video no arabic subtitles")
                return False, False, False
            
        except Exception as e:
             print(e)
             return None,None, None


# # Function to cut video into generated video segments
# def cut_video_into_segments(video_path, segment_length=2):

#     # cut the segments into similary named parts as the parent

#     clip = VideoFileClip(video_path)
#     duration = int(clip.duration)
#     for start in range(0, duration, segment_length):
#         end = min(start + segment_length, duration)
#         segment = clip.subclip(start, end)
#         segment_filename = f"output_segment_{start}_{end}.mp4"
#         segment.write_videofile(segment_filename, codec="libx264", audio_codec="aac")
        
def english_to_arabic_numbers(text):

    numbers_dict = {
        '0': '٠',
        '1': '١',
        '2': '٢',
        '3': '٣',
        '4': '٤',
        '5': '٥',
        '6': '٦',
        '7': '٧',
        '8': '٨',
        '9': '٩',
    }
    for eng_num, arabic_num in numbers_dict.items():
        text = text.replace(eng_num, arabic_num)
    return text

def remove_special_char(text):
    for i in string.punctuation:
        text = text.replace(i, " ")

    return text

def remove_repeating_spaces(text):
    cleaned_text = re.sub(r' +', ' ', text)
    return cleaned_text

def check_if_not_arabic(text):
    ar = "ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩ "
    for i in text:
        if i not in ar:
            return True
    return False

def filter_srt_sub(text):
    
    if(len(text) < 2):
        return False

    if(text == "[موسيقى]"):
        return False
    
    text = english_to_arabic_numbers(text)
    text = remove_special_char(text)
    text = remove_repeating_spaces(text)
    
    if(check_if_not_arabic(text)):
        return False
    
    if(len(text) < 2):
        return False
    
    return text

def filter_lip_segment():
    pass

def cut_from_srt_file(video_path, srt_path, output_dir_videos,output_dir_str, id):
    
    subs = pysrt.open(srt_path)
    video = VideoFileClip(video_path)

    os.makedirs(output_dir_videos, exist_ok=True)

    for idx, sub in enumerate(subs):
        text = filter_srt_sub(sub.text.strip())

        if( not text):
            continue

        start_time = sub.start.ordinal /1000.0
        end_time = sub.end.ordinal / 1000     

        video_part = video.subclip(start_time, end_time)
        output_path = os.path.join(output_dir_videos, f"{id}_{idx+1}.mp4")
        
        with open(output_dir_str+f"/{id}_{idx+1}.align","w") as writer:
            writer.write(text.replace(" ","\n"))

        video_part.write_videofile(output_path)


    video.close()

def video_is_accpeted(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None

    mp_face_detection = mp.solutions.face_detection
    if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) > 300):
        return False
    
    # frames of the video
    # frames = np.empty((max_frames, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
# discard when
# orentation of face not in camera range  for 0.8 of frames 

# cut when
# start for massive deviation of lips movment, end when end of deviation
    # idx = 0
    bad_face = False
    with mp_face_detection.FaceDetection(
        model_selection=2, min_detection_confidence=1
    ) as face_detection:
        # loop over all frames in the video
        for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            ret, frame = cap.read()

            # convert to rgb
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # get the keypoints (x,y)s of each point of a face in the frame
            results = face_detection.process(frame)
            if results.detections:
                if(len(results.detections) !=1):
                    return False
            else:
                return False
            
            # check orientation



            # check if lips closed


            # loop over each face landmark ~ (nose (x,y) lips (x,y) and so one and a bounding box)
            # if results.detections:
            #     for detection in results.detections:
            #         location = detection.location_data


    return True
    pass

def filter_videos(videos_path,alignments_path,output_videos,output_alignments,id):
    for i in glob.glob(videos_path+fr"/{id}*.mp4"):
        video_path = i.replace("\\","/")
        alignment_path = alignments_path+"/"+re.split(r"\\|/",i)[-1].split(".")[0]+".align"
        output_video_path = output_videos+"/"+re.split(r"\\|/",i)[-1]
        output_alignment_path = output_alignments+"/"+re.split(r"\\|/",i)[-1].split(".")[0]+".align"

        if(video_is_accpeted(video_path)):
            shutil.copy(video_path, output_video_path)
            shutil.copy(alignment_path, output_alignment_path)
            

# Function to filter unacceptable video segments

# create segments
# for loop on all the segments



In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Face and Pose modules
mp_face = mp.solutions.face_detection
mp_pose = mp.solutions.pose

# Initialize the MediaPipe drawing utilities
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)  # Use the desired camera index (e.g., 0 for the default camera)

# Initialize Face Detection and Pose Estimation models
face_detection = mp_face.FaceDetection(min_detection_confidence=0.5)
pose_estimation = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
while True:
    success, frame = cap.read()
    if not success:
        continue

    # Convert the frame to RGB format for MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform face detection
    face_results = face_detection.process(frame_rgb)

    if face_results.detections:
        for detection in face_results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Perform pose estimation using the face region
            face_center = (x + w // 2, y + h // 2)
            frame_pose = frame[y:y + h, x:x + w]
            frame_pose_rgb = cv2.cvtColor(frame_pose, cv2.COLOR_BGR2RGB)

            # Perform pose estimation on the face region
            pose_results = pose_estimation.process(frame_pose_rgb)

            if pose_results.pose_landmarks:
                # You can access pose landmarks and orientation information here
                landmarks = pose_results.pose_landmarks
                # Extract relevant pose landmarks and calculate face orientation

                # Draw landmarks and orientation lines
                mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Face Orientation Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Mgala\AppData\Roaming\Python\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [59]:
# cut_from_srt_file(r"data\Video Links Dataset\temp_videos\0.mp4", r"data\Video Links Dataset\temp_alignments\0 (a.ar).srt.fixed", 
#                   r"data\Video Links Dataset\temp_videos",r"data\Video Links Dataset\temp_alignments", 0)

In [60]:
filter_videos(r"data/Video Links Dataset/temp_videos", r"data/Video Links Dataset/temp_alignments", 
                  r"data/Video Links Dataset/videos",r"data/Video Links Dataset/alignments", 0)

In [5]:
def pipline(url, name):
    fails = 10

    video, alignment, id = download_video(url,name)
    while (video == None) and (fails > 0):
        fails -= 1
        sleep(4)
        video, alignment, id = download_video(url,name)

    if not video:
        return False
    print("="*100)
    print(alignment)
    print("="*100)
    subprocess.run(["go","run",".",alignment])

    cut_from_srt_file(video,alignment+".fixed",r"data/Video Links Dataset/temp_videos",r"data/Video Links Dataset/temp_alignments", id)
    os.remove(video)
    os.remove(alignment)
    filter_videos(r"data/Video Links Dataset/temp_videos",r"data/Video Links Dataset/temp_alignments",r"data/Video Links Dataset/videos",r"data/Video Links Dataset/alignments")
    return True
    

In [6]:
df = pd.read_csv("data\Video Links Dataset\Video Links Dataset.csv")
df.head()

,link,name,id,creator
0,https://www.youtube.com/watch?v=s0pbzSC81Kc,كتابة الكلام على الفيديو بشكل تلقائي بالذكاء ا...,0,0
1,https://www.youtube.com/watch?v=FYqNG3zwXL4&t=...,فيلم الصفاره,1,0
2,https://www.youtube.com/watch?v=cAsTfosN79A,فيلم كابتن مصر,2,0
3,https://www.youtube.com/watch?v=SJ6_7W2v8ac,فيلم التجربة الدنماركيه,3,0
4,https://www.youtube.com/watch?v=3r3IOzNNKRM,جت كدا اياد الموجي وساره هاني,4,0


In [7]:
df[df.duplicated('link',keep=False)]

,link,name,id,creator


In [8]:
results = []

In [10]:
status = []

# downloaded
# No CC
# Failed

for row in df.values:
     
     result = pipline(row[0], row[2])
     results.append(result)
     break
df['has_arabic'] = status

<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
data/Video Links Dataset/temp_alignments/0 (a.ar).srt


AttributeError: 'SubRipItem' object has no attribute 'strip'

In [ ]:
# code in captions.py
    # def xml_caption_to_srt(self, xml_captions: str) -> str:
    #     """Convert xml caption tracks to "SubRip Subtitle (srt)".

    #     :param str xml_captions:
    #     XML formatted caption tracks.
    #     """
    #     segments = []
    #     root = ElementTree.fromstring(xml_captions)
    #     i=0
    #     for child in list(root.iter("body"))[0]:
    #         if child.tag == 'p':
    #             caption = ''
    #             if len(list(child))==0:
    #                 # instead of 'continue'
    #                 caption = child.text
    #             for s in list(child):
    #                 if s.tag == 's':
    #                     caption += ' ' + s.text
    #             caption = unescape(caption.replace("\n", " ").replace("  ", " "),)
    #             try:
    #                 duration = float(child.attrib["d"])/1000.0
    #             except KeyError:
    #                 duration = 0.0
    #             start = float(child.attrib["t"])/1000.0
    #             end = start + duration
    #             sequence_number = i + 1  # convert from 0-indexed to 1.
    #             line = "{seq}\n{start} --> {end}\n{text}\n".format(
    #                 seq=sequence_number,
    #                 start=self.float_to_srt_time_format(start),
    #                 end=self.float_to_srt_time_format(end),
    #                 text=caption,
    #             )
    #             segments.append(line)
    #             i += 1
    #     return "\n".join(segments).strip()